In [9]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset
data = pd.read_csv('SA.csv', encoding='latin-1', header=None)
data.columns = ['polarity', 'id', 'date', 'query', 'user', 'text']

In [10]:
# Clean the text data
def clean_text(text):
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\W', ' ', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

data['cleaned_text'] = data['text'].apply(clean_text)




In [11]:
# Encode labels
label_encoder = LabelEncoder()
data['polarity'] = label_encoder.fit_transform(data['polarity'])

# Split the data
X_train, X_temp, y_train, y_temp = train_test_split(data['cleaned_text'], data['polarity'], test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [12]:
import keras
print(keras.__version__)


3.6.0


In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [15]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# Tokenization
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding sequences
max_length = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length)

# LSTM Model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_length))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


C:\Users\majlenovo\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [16]:
history = model.fit(X_train_pad, y_train, validation_data=(X_val_pad, y_val), epochs=5, batch_size=64)

Epoch 1/5
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 3129s 178ms/step - accuracy: 0.7887 - loss: 0.4489 - val_accuracy: 0.8179 - val_loss: 0.3993
Epoch 2/5
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 1741s 99ms/step - accuracy: 0.8235 - loss: 0.3895 - val_accuracy: 0.8242 - val_loss: 0.3880
Epoch 3/5
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 2988s 171ms/step - accuracy: 0.8342 - loss: 0.3700 - val_accuracy: 0.8272 - val_loss: 0.3839
Epoch 4/5
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 2129s 122ms/step - accuracy: 0.8430 - loss: 0.3536 - val_accuracy: 0.8274 - val_loss: 0.3841
Epoch 5/5
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 3473s 198ms/step - accuracy: 0.8504 - loss: 0.3395 - val_accuracy: 0.8267 - val_loss: 0.3899


In [17]:
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test)
print(f'Test Accuracy: {test_accuracy:.4f}')

7500/7500 ━━━━━━━━━━━━━━━━━━━━ 175s 23ms/step - accuracy: 0.8291 - loss: 0.3836
Test Accuracy: 0.8270


In [18]:
from sklearn.metrics import classification_report

y_pred = (model.predict(X_test_pad) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))

7500/7500 ━━━━━━━━━━━━━━━━━━━━ 150s 20ms/step
              precision    recall  f1-score   support

           0       0.83      0.83      0.83    119907
           1       0.83      0.83      0.83    120093

    accuracy                           0.83    240000
   macro avg       0.83      0.83      0.83    240000
weighted avg       0.83      0.83      0.83    240000

